In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORT THE LIBRARIES

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,f1_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve
from sklearn.model_selection import cross_val_score,KFold,cross_val_score,cross_validate
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.tree import plot_tree
from xgboost import plot_tree,plot_importance
from sklearn import tree
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from yellowbrick.classifier import ClassificationReport, confusion_matrix,precision_recall_curve,PrecisionRecallCurve,ConfusionMatrix,ROCAUC,DiscriminationThreshold
from yellowbrick.model_selection import FeatureImportances,learning_curve,LearningCurve
from yellowbrick.target import class_balance
import itertools
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from lightgbm import LGBMClassifier
from scipy.stats import boxcox
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,KMeansSMOTE
from imblearn.under_sampling import NearMiss,RandomUnderSampler
import h2o
from h2o.automl import H2OAutoML
!pip install researchpy
import researchpy

# EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
data=pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df=data.copy()
df.head()

In [ ]:
print("Number of Columns :{0} \nNumber of Rows : {1}".format(df.shape[0],df.shape[1]))

In [ ]:
df.dtypes

In [ ]:
df.loc[df.TotalCharges==" ","TotalCharges"]=np.nan

In [ ]:
df.TotalCharges=df.TotalCharges.astype(float)

In [ ]:
df.describe().T

In [ ]:
df.SeniorCitizen=df["SeniorCitizen"].replace({0:"No",1:"Yes"})

In [ ]:
df.describe(include=["object"]).T

In [ ]:
def diagnostic_plots(df, variable):
    
    plt.figure(figsize=(20, 9))

    plt.subplot(1, 3, 1)
    sns.distplot(df[variable], bins=30,kde_kws={'bw': 1.5})
    plt.title('Histogram')
    
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('RM quantiles')

    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')
    
    plt.show()

In [ ]:
num_columns=df.select_dtypes(exclude=["object"]).columns
for i in num_columns:
    diagnostic_plots(df,i)


In [ ]:
plt.figure(figsize=(12,5))
corr=df.corr().abs()
mask=np.zeros_like(corr,dtype=np.bool)
mask[np.triu_indices_from(mask)]=True
sns.heatmap(corr,annot=True,cmap="coolwarm",mask=mask);

In [ ]:
researchpy.correlation.corr_pair(df.select_dtypes(exclude="object"))

In [ ]:
for i in df.select_dtypes(include="object").columns[1:]:
    print(str(i) + " and " + "Churn")
    crosstab, res = researchpy.crosstab(df[i], df["Churn"], test= "chi-square")
    print(res);

In [ ]:
sns.pairplot(df,hue="Churn",aspect=3);

In [ ]:
sns.FacetGrid(df,hue="Churn",height=5,aspect=2).map(sns.kdeplot,"TotalCharges",shade=False).add_legend();

In [ ]:
sns.FacetGrid(df,hue="Churn",height=5,aspect=2).map(sns.kdeplot,"tenure",shade=False).add_legend();

In [ ]:
sns.FacetGrid(df,hue="Churn",height=5,aspect=2).map(sns.kdeplot,"MonthlyCharges",shade=False).add_legend();

In [ ]:
cat_columns=df.select_dtypes(include="object").columns[1:]
for i in cat_columns[:-1]:
    sns.catplot(x=i,y="TotalCharges",hue="Churn",data=df,kind="box",aspect=2);

In [ ]:
df.info()

In [ ]:
class_balance(df.iloc[:,-1]);

In [ ]:

for i in cat_columns:
    plt.figure(figsize=(12,5));
    sns.countplot(df[i],hue="Churn",data=df)
    plt.show()

In [ ]:
df.isnull().sum()/(len(df))

In [ ]:
df[pd.isnull(df["TotalCharges"])]

In [ ]:
df.TotalCharges.fillna(df.TotalCharges.median(),inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df=df.drop(columns=["customerID"],axis=1)

In [ ]:
df_coded=pd.get_dummies(df,drop_first=True)

In [ ]:
X=df_coded.iloc[:,:-1]
y=df_coded.iloc[:,-1]

# TRAIN TEST SPLIT

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

In [ ]:

class_balance(y_train);
class_balance(y_test);

# MODELLING

In [ ]:
models = []
models.append(('LR', LogisticRegression(random_state = 12345)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(random_state = 12345)))
models.append(('RF', RandomForestClassifier(random_state = 12345)))
#models.append(('SVC', SVC(gamma='auto', random_state = 12345)))
models.append(("LightGBM", LGBMClassifier(random_state = 12345)))
models.append(("XGBoost", XGBClassifier(random_state = 12345)))
models.append(("GBM", GradientBoostingClassifier(random_state = 12345)))
models.append(("HGBoost", HistGradientBoostingClassifier(random_state = 12345)))
models.append(("Adaboost", AdaBoostClassifier(random_state = 12345)))
models.append(("Gaussian", GaussianNB()))
#models.append(("Discriminant", QuadraticDiscriminantAnalysis()))
#models.append(("Catboost", CatBoostClassifier(random_state=12345)))

In [ ]:
results = []
names = []

In [ ]:
for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 12345)
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
        


In [ ]:


def visualize_result(model):
    print("*"*5 + str(model) + "*"*5)
    fig, axes = plt.subplots(2, 2,figsize=(20,10))

    model = model;
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Accuracy Score :",accuracy_score(y_test,y_pred))
    print("Matthews_Corr :",matthews_corrcoef(y_test,y_pred))
    print("F1_Score :",f1_score(y_test,y_pred))
    
    visualgrid = [
        ConfusionMatrix(model, ax=axes[0][0],cmap="YlGn"),
        PrecisionRecallCurve(model,ax=axes[0][1]),  
        #FeatureImportances(model,ax=axes[0][0]),        
        ClassificationReport(model, ax=axes[1][0]),
        ROCAUC(model, ax=axes[1][1])
    ]

    for viz in visualgrid:
        
        viz.fit(X_train, y_train)
        viz.score(X_test, y_test)
        viz.finalize()


    plt.show()
    

In [ ]:
for i in range(len(models)):    
    visualize_result(models[i][1])

# UNDERSAMPLING

## NearMiss

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=42)
nm = NearMiss()
X_train, y_train = nm.fit_sample(X_train, y_train)
y_train = y_train.astype(int)

In [ ]:
class_balance(y_train);
class_balance(y_test);

In [ ]:
for i in range(len(models)):    
    visualize_result(models[i][1])

## RandomUnderSampler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=42)
rus = RandomUnderSampler(random_state=42)
X_train,y_train = rus.fit_resample(X_train, y_train)

for i in range(len(models)):    
    visualize_result(models[i][1])

# OVERSAMPLING

## SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=42)
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)
y_train = y_train.astype(int)

In [ ]:
class_balance(y_train);
class_balance(y_test);

In [ ]:
for i in range(len(models)):    
    visualize_result(models[i][1])

# NEW MODEL

In [ ]:
for i in df.select_dtypes(include=["object"]).columns:  
    print("-"*5 + str(i) + "-"*5)
    print(df[i].value_counts())  

In [ ]:
#MultipleLines ,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamigTV

df_new=df.replace({"No internet service" : "No","No phone service":"No"})
df_new=pd.get_dummies(df_new,drop_first=True)


In [ ]:
df_new.head()

In [ ]:
plt.subplot(121)
sns.distplot(np.array(stats.boxcox(df_new.TotalCharges))[0]);
plt.title("Box Cox Transformation");
plt.subplot(122)
sns.distplot(np.log(df_new.TotalCharges));
plt.title("Log Transformation");

In [ ]:
df_scaled=df_new.copy()
df_scaled.TotalCharges=np.array(stats.boxcox(df_new.TotalCharges))[0]

In [ ]:
scaler=MinMaxScaler()
df_scaled=scaler.fit_transform(df_new)
df_scaled=pd.DataFrame(df_scaled,columns=df_new.columns,index=df_new.index)

In [ ]:
df_scaled.head()

In [ ]:
X=df_scaled.iloc[:,:-1]
y=df_scaled.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=42)

In [ ]:
for i in range(len(models)):    
    visualize_result(models[i][1])

# UNDERSAMPLING ON NEW MODEL

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=42)
rus = RandomUnderSampler()
X_train, y_train = rus.fit_sample(X_train, y_train)
y_train = y_train.astype(int)

for i in range(len(models)):    
    visualize_result(models[i][1])

# OVERSAMPLING ON NEW MODEL

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20,random_state=42)
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)
y_train = y_train.astype(int)

In [ ]:
for i in range(len(models)):    
    visualize_result(models[i][1])

# H2O AUTO ML

In [ ]:
h2o.init()

In [ ]:
df_h2o=data.copy()
df_h2o.loc[df_h2o.TotalCharges==" ","TotalCharges"]=np.nan
df_h2o.TotalCharges=df_h2o.TotalCharges.astype(float)
df_h2o.SeniorCitizen=df_h2o["SeniorCitizen"].replace({0:"No",1:"Yes"})

In [ ]:
df_h2o=h2o.H2OFrame(df_h2o)
df_h2o.head()

In [ ]:
df_h2o["Churn"].table()

In [ ]:
split=df_h2o.split_frame(ratios=[0.8],seed=12345)
train=split[0]
test=split[1]
train.shape,test.shape

In [ ]:
train["Churn"].table()

In [ ]:
test["Churn"].table()

In [ ]:
x = train.columns
y = "Churn"
x.remove(y)

In [ ]:
aml = H2OAutoML(max_models=25, seed=1,balance_classes=True)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
aml.leader

In [ ]:
preds = aml.predict(test)
#preds = aml.leader.predict(test)

In [ ]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
lb

In [ ]:
perf = aml.leader.model_performance(test)
perf.auc()